# Each Process (Removing qubit controls and adjacent gates pairs)

In slimming process, first unnecessary qubit controls are removed and then adjacent gates pairs are removed. This is the sub-routine, so this routine is repeated until there are no changes in the circuit by slimming.

In [1]:
import numpy as np
from qiskit import *
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit.library.standard_gates import CXGate, CCXGate, XGate, CRYGate, RYGate, MCXGate, U3Gate, CU3Gate
import copy
from qiskit.circuit import Qubit
from qiskit.tools.monitor import job_monitor
#from operator import length_hint

# Import general libraries (needed for functions)
import numpy as np
import time

class circuit_optimization():
    
    def __init__(self, circuit, slim_level, cut, work_register='None'):
        self.circuit = circuit
        self.slim_level = slim_level
        self.dag = circuit_to_dag(self.circuit)
        self.work_register = work_register
        self.cut = cut
    
    def slim(self):
        t1 = time.time()
        
        noU3_dag   = self.u3_transfer_ry(self.dag)
        t2 = time.time()
        
        delete_dag = self.delete(noU3_dag)
        while delete_dag != self.delete(delete_dag):
            delete_dag = self.delete(delete_dag)
        t3 = time.time()
        print('delete1 time:',t3-t2)
        
        basis_dag  = self.basis(delete_dag)
        t4 = time.time()
        print('basis time:',t4-t3)
        
        delete_dag = self.delete(basis_dag)
        while delete_dag != self.delete(delete_dag):
            delete_dag = self.delete(delete_dag)
        t5 = time.time()
        print('delete2 time:',t5-t4)
        
        circuit = dag_to_circuit(delete_dag)
        t6 = time.time()
        
        circuit = self.delete_qubit(circuit)
        t7 = time.time()
        print('delete qubit time:',t7-t6)
        
        print('all time:',t7-t1)
        
        return circuit

    def u3_transfer_ry(self,dag):
        for x in range(len(dag.op_nodes())):
            if dag.op_nodes()[x].name == 'u3':
                if (dag.op_nodes()[x].op.params[1]==0) & (dag.op_nodes()[x].op.params[2]==0):
                    theta=dag.op_nodes()[x].op.params[0]
                    self.u3_to_ry(dag,theta)
                    
            if dag.op_nodes()[x].name == 'cu3':
                if (dag.op_nodes()[x].op.params[1]==0) & (dag.op_nodes()[x].op.params[2]==0):
                    theta=dag.op_nodes()[x].op.params[0]
                    self.cu3_to_cry(dag,theta)             
        return dag

    def delete(self,dag):
        len_dag=len(dag.op_nodes())
        delete_list=[]
        i=0
        for x in range(len_dag-1): # target gate judged whether it can be removed or not
            if x-i>0:
                x=x-i
            else:
                x=0
            if dag.op_nodes()[x].name != 'barrier': #barrier is ignored
                for z in range(len_dag-x-1): #compared gate with target gate
                    for a in range(len(dag.op_nodes()[x].qargs)):
                        for b in range(len(dag.op_nodes()[x+z+1].qargs)):
                            if (dag.op_nodes()[x].qargs[-1]==dag.op_nodes()[x+z+1].qargs[b]) or (dag.op_nodes()[x].qargs[a]==dag.op_nodes()[x+z+1].qargs[-1]): #find the nearest gate that share a same qubit which is a target qubit
                                if (dag.op_nodes()[x].name == 'cry') or (dag.op_nodes()[x].name == 'ry'): 
                                    if (dag.op_nodes()[x].name==dag.op_nodes()[x+z+1].name) & (dag.op_nodes()[x].qargs==dag.op_nodes()[x+z+1].qargs):
                                        if dag.op_nodes()[x].op.params[0] == -(dag.op_nodes()[x+z+1].op.params[0]):
                                            dag.remove_op_node(dag.op_nodes()[x])
                                            dag.remove_op_node(dag.op_nodes()[x+z])
                                            len_dag=len_dag-2
                                            i+=2
                                        else:
                                            dag.op_nodes()[x+z+1].op.params[0] = dag.op_nodes()[x+z+1].op.params[0]+dag.op_nodes()[x].op.params[0]
                                            dag.remove_op_node(dag.op_nodes()[x])
                                            len_dag=len_dag-1
                                            i+=1
                                elif (dag.op_nodes()[x].name==dag.op_nodes()[x+z+1].name) & (dag.op_nodes()[x].qargs==dag.op_nodes()[x+z+1].qargs):
                                        dag.remove_op_node(dag.op_nodes()[x])
                                        dag.remove_op_node(dag.op_nodes()[x+z])
                                        len_dag=len_dag-2
                                        i+=2
                                break   
                        else:
                            continue
                        break
                    else:
                        continue
                    break

        return dag
  
    def basis(self,dag):
        node_list=[]
        remove_list=[]
        ccx_counter=0
        cx_counter =0
        cry_counter=0
        basis_info=[[0]*dag.num_qubits()]   # Indexes in this list matches indexes of qubits in the circuit

        
        barrier_count=0
        for x in range(len(dag.op_nodes())):
            
            if dag.op_nodes()[x].name == 'barrier':
                barrier_count+=1
                
            if barrier_count % 2 == 0: # Remove qubit controls except recurring gates

                fig_list=[1] # This list includes the information of indexes of controlled qubits and target qubits. First "1" is nothing. Later I will remove)

                for a in dag.op_nodes()[x].qargs:
                    for n in range(self.circuit.num_qubits):
                        if a == self.circuit.qubits[n]:
                            fig_list.append(n)

                bit_state={} # List of used computational bases on the controlled qubits
                for basis in basis_info:
                    bit_state.setdefault(tuple([basis[fig_list[i+1]] for i in range(len(fig_list)-2)]),1)

                # Remove unnecessary qubit controls
                if dag.op_nodes()[x].name == 'cx':
                    if (((1,) in bit_state) == True) & (((0,) in bit_state) == False):
                        self.cx_to_x(cx_counter,dag)
                        cx_counter=cx_counter-1
                    elif ((1,) in bit_state) == False:
                        remove_list.append(x)
                    cx_counter+=1

                elif dag.op_nodes()[x].name == 'ccx':
                    if ((1,1) in bit_state) == True:
                        if len(bit_state) == 1:
                            self.ccx_to_x(ccx_counter,dag)
                            ccx_counter=ccx_counter-1
                        elif ((0,1) in bit_state) == False:
                            self.ccx_to_cx(ccx_counter,dag,1)
                            ccx_counter=ccx_counter-1
                            cx_counter+=1
                        elif ((1,0) in bit_state) == False:
                            self.ccx_to_cx(ccx_counter,dag,0)
                            ccx_counter=ccx_counter-1
                            cx_counter+=1                  
                    else:
                        remove_list.append(x)
                    ccx_counter+=1

                elif dag.op_nodes()[x].name == 'cry':
                    if (((1,) in bit_state) == True) & (((0,) in bit_state) == False):
                        self.cry_to_ry(cry_counter, dag, dag.op_nodes()[x].op.params[0])
                        cry_counter=cry_counter-1
                    elif ((1,) in bit_state) == False:
                        remove_list.append(x)
                    cry_counter+=1
            
            if barrier_count % 2 != 0:
                if dag.op_nodes()[x].name == 'cx':
                    cx_counter+=1
                elif dag.op_nodes()[x].name == 'ccx':
                    ccx_counter+=1
                elif dag.op_nodes()[x].name == 'cry':
                    cry_counter+=1
                
            
            # Identify used computational bases list
            fig_list=[1]
            
            for a in dag.op_nodes()[x].qargs:
                for n in range(self.circuit.num_qubits):
                    if a == self.circuit.qubits[n]:
                        fig_list.append(n)

            new_basis=[]
            
            for basis in basis_info:

                if dag.op_nodes()[x].name == 'x':
                    basis[fig_list[1]]=int(not basis[fig_list[1]])
                
                elif (dag.op_nodes()[x].name == 'cx') or (dag.op_nodes()[x].name == 'ccx'):
                    if [basis[fig_list[i+1]] for i in range(len(fig_list)-2)] == [1]*(len(fig_list)-2):
                        basis[fig_list[-1]]=int(not basis[fig_list[-1]])
                    elif basis[fig_list[-1]] == 1:
                        basis[fig_list[-1]]=1
                    else:
                        basis[fig_list[-1]]=0

                elif (dag.op_nodes()[x].name == 'h') or (dag.op_nodes()[x].name == 'ry'):
                    basis2=copy.copy(basis)
                    basis2[fig_list[1]]=int(not basis[fig_list[1]])
                    new_basis.append(basis2)
                    
                elif dag.op_nodes()[x].name == 'cry':
                    if basis[fig_list[1]] == 1:
                        basis2=copy.copy(basis)
                        basis2[fig_list[2]]=int(not basis[fig_list[2]])
                        new_basis.append(basis2)
                    elif basis[fig_list[2]] == 1:
                        basis[fig_list[2]]=1
                    else:
                        basis[fig_list[2]]=0

            
            for basis in new_basis:
                basis_info.append(basis)    
            
            basis_info = self.get_unique_list(basis_info)

        i=0
        for x in remove_list:
            dag.remove_op_node(dag.op_nodes()[x-i])
            i=i+1
            
        return dag

    # Remove unused qubits
    def delete_qubit(self,circ):
    
        for _ in range(circ.num_qubits):
            
            c_register_list={}
            for clbit in circ.clbits:
                c_register_list.setdefault(clbit.register,1)
                
            new_c_list=[]
            for key in c_register_list:
                new_c_list.append(key)
            
            register_list={}
            for qubit in circ.qubits:
                register_list.setdefault(qubit.register,1)

            qubit_list={}
            for gate in circ:
                if gate[0].name != 'barrier':
                    for qubit in gate[1]:
                        qubit_list.setdefault(qubit,1)

            for qubit in circ.qubits:
                if (qubit in qubit_list) == False:
                    delete_index = qubit.index
                    delete_register = qubit.register

                    if delete_register in register_list:
                        register_list.pop(delete_register)

                    if qubit.register.size > 1:
                        new_register = QuantumRegister(qubit.register.size-1, qubit.register.name)
                        register_list.setdefault(new_register,1)

                    gate_list=[]
                    for gate in circ:
                        gate_list.append(list(gate))
                        
                    for gate in gate_list:
                        if gate[0].name == 'barrier':
                            if len(gate[1])==1:
                                if (gate[1][x].register == delete_register) & (gate[1][x].index == delete_index):
                                        gate_list.pop(gate_list.index(gate))
                            if len(gate[1])>1:
                                b_qubit_list=[]
                                for qubit in gate[1]:
                                    if qubit != Qubit(delete_register, delete_index):
                                        b_qubit_list.append(qubit)
                                gate[1] = b_qubit_list

                    for gate in gate_list:
                        for x in range(len(gate[1])):
                            if gate[1][x].register == delete_register:
                                if gate[1][x].index < delete_index:
                                    gate[1][x]=Qubit(new_register, gate[1][x].index)
                                else:
                                    gate[1][x]=Qubit(new_register, gate[1][x].index-1)

                    new_list=[]
                    for key in register_list:
                        new_list.append(key)
                    circ = QuantumCircuit(*new_list,*new_c_list)
                    for gate in gate_list:
                        circ.append(gate[0],gate[1],gate[2])
        return circ
    

    @staticmethod
    def ccx_to_cx(ccx_counter,dag,controlled_index):
        mini_dag = DAGCircuit()
        p = QuantumRegister(3, "p")
        mini_dag.add_qreg(p)
        mini_dag.apply_operation_back(CXGate(), qargs=[p[1],p[2]])
        # substitute the cx node with the above mini-dag
        ccx_node = dag.op_nodes(op=CCXGate).pop(ccx_counter)

        if controlled_index == 0: 
            dag.substitute_node_with_dag(node=ccx_node, input_dag=mini_dag, wires=[p[1], p[0], p[2]])
        if controlled_index == 1:
            dag.substitute_node_with_dag(node=ccx_node, input_dag=mini_dag, wires=[p[0], p[1], p[2]])
        return dag
    
    @staticmethod
    def ccx_to_x(ccx_counter,dag):
        mini_dag = DAGCircuit()
        p = QuantumRegister(3, "p")
        mini_dag.add_qreg(p)
        mini_dag.apply_operation_back(XGate(), qargs=[p[2]])
        # substitute the cx node with the above mini-dag
        ccx_node = dag.op_nodes(op=CCXGate).pop(ccx_counter)
        dag.substitute_node_with_dag(node=ccx_node, input_dag=mini_dag, wires=[p[0], p[1], p[2]])
        return dag

    @staticmethod
    def cx_to_x(cx_counter,dag):
        mini_dag = DAGCircuit()
        p = QuantumRegister(2, "p")
        mini_dag.add_qreg(p)
        mini_dag.apply_operation_back(XGate(), qargs=[p[0]])
        # substitute the x node with the above mini-dag
        cx_node = dag.op_nodes(op=CXGate).pop(cx_counter)
        dag.substitute_node_with_dag(node=cx_node, input_dag=mini_dag, wires=[p[1], p[0]])
        return dag

    @staticmethod
    def cry_to_ry(cry_counter,dag,theta):
        mini_dag = DAGCircuit()
        p = QuantumRegister(2, "p")
        mini_dag.add_qreg(p)
        mini_dag.apply_operation_back(RYGate(theta), qargs=[p[0]])
        # substitute the ry node with the above mini-dag
        cry_node = dag.op_nodes(op=CRYGate).pop(cry_counter)
        dag.substitute_node_with_dag(node=cry_node, input_dag=mini_dag, wires=[p[1], p[0]])
        return dag

    @staticmethod
    def u3_to_ry(dag,theta):
        mini_dag = DAGCircuit()
        p = QuantumRegister(1, "p")
        mini_dag.add_qreg(p)
        mini_dag.apply_operation_back(RYGate(theta), qargs=[p[0]])
        # substitute the ry node with the above mini-dag
        u3_node = dag.op_nodes(op=U3Gate).pop(0)
        dag.substitute_node_with_dag(node=u3_node, input_dag=mini_dag, wires=[p[0]])
        return dag
    
    @staticmethod
    def cu3_to_cry(dag,theta):
        mini_dag = DAGCircuit()
        p = QuantumRegister(2, "p")
        mini_dag.add_qreg(p)
        mini_dag.apply_operation_back(CRYGate(theta), qargs=[p[0],p[1]])
        # substitute the ry node with the above mini-dag
        cu3_node = dag.op_nodes(op=CU3Gate).pop(0)
        dag.substitute_node_with_dag(node=cu3_node, input_dag=mini_dag, wires=[p[0],p[1]])
        return dag

    @staticmethod
    def get_unique_list(seq):
        seen = []
        return [x for x in seq if x not in seen and not seen.append(x)]

# PS 1step

## No_slimming

In [2]:
from sample_algorithm.onestepSim_LBNL import runQuantum
circuit_LBNL1 = runQuantum(gLR=1,dophisplit=1)

In [3]:
print(circuit_LBNL1.depth(), ',', circuit_LBNL1.__len__())
print('Gate counts:', circuit_LBNL1.count_ops())

46 , 66
Gate counts: OrderedDict([('ccx', 28), ('x', 19), ('cu3', 7), ('measure', 6), ('cx', 5), ('barrier', 1)])


In [4]:
circuit_LBNL1_basis = circuit_LBNL1.decompose()
print(circuit_LBNL1_basis.depth(), ',', circuit_LBNL1_basis.__len__())
print('Gate counts:', circuit_LBNL1_basis.count_ops())

309 , 493
Gate counts: OrderedDict([('cx', 187), ('t', 112), ('tdg', 84), ('h', 56), ('u3', 33), ('u1', 14), ('measure', 6), ('barrier', 1)])


## Removing adjacent gates pairs

In [5]:
from qiskit.converters import circuit_to_dag, dag_to_circuit

In [6]:
from transpiler.optimization import slim
example1 = slim.circuit_optimization( circuit=circuit_LBNL1, slim_level=1, work_register = 'w', cut='high')

In [7]:
circuit_LBNL1 = runQuantum(gLR=1,dophisplit=1)
dag1=circuit_to_dag(circuit_LBNL1)
noU3_dag=example1.u3_transfer_ry(dag1)

In [8]:
delete_dag1 = example1.delete(noU3_dag)
slimmed_delete1 = dag_to_circuit(delete_dag1)

In [9]:
print(slimmed_delete1.depth(), ',', slimmed_delete1.__len__())
print('Gate counts:', slimmed_delete1.count_ops())

44 , 56
Gate counts: OrderedDict([('ccx', 28), ('x', 9), ('cry', 7), ('measure', 6), ('cx', 5), ('barrier', 1)])


In [10]:
slimmed_delete1_basis = slimmed_delete1.decompose()
print(slimmed_delete1_basis.depth(), ',', slimmed_delete1_basis.__len__())
print('Gate counts:', slimmed_delete1_basis.count_ops())

300 , 469
Gate counts: OrderedDict([('cx', 187), ('t', 112), ('tdg', 84), ('h', 56), ('u3', 23), ('measure', 6), ('barrier', 1)])


## Removing qubit controls (only)

In [11]:
basis_dag1 = example1.basis(delete_dag1)
slimmed_ctrl1 = dag_to_circuit(basis_dag1)

In [12]:
print(slimmed_ctrl1.depth(), ',', slimmed_ctrl1.__len__())
print('Gate counts:', slimmed_ctrl1.count_ops())

43 , 55
Gate counts: OrderedDict([('cx', 29), ('x', 9), ('measure', 6), ('cry', 4), ('ccx', 4), ('ry', 2), ('barrier', 1)])


In [13]:
slimmed_ctrl1_basis = slimmed_ctrl1.decompose()
print(slimmed_ctrl1_basis.depth(), ',', slimmed_ctrl1_basis.__len__())
print('Gate counts:', slimmed_ctrl1_basis.count_ops())

83 , 123
Gate counts: OrderedDict([('cx', 61), ('u3', 17), ('t', 16), ('tdg', 12), ('h', 8), ('measure', 6), ('r', 2), ('barrier', 1)])


## Removing adjacent gates pairs

In [14]:
delete_dag2 = example1.delete(basis_dag1)
slimmed_delete2 = dag_to_circuit(delete_dag2)

In [15]:
print(slimmed_delete2.depth(), ',', slimmed_delete2.__len__())
print('Gate counts:', slimmed_delete2.count_ops())

43 , 53
Gate counts: OrderedDict([('cx', 29), ('x', 7), ('measure', 6), ('cry', 4), ('ccx', 4), ('ry', 2), ('barrier', 1)])


In [16]:
slimmed_delete2_basis = slimmed_delete2.decompose()
print(slimmed_delete2_basis.depth(), ',', slimmed_delete2_basis.__len__())
print('Gate counts:', slimmed_delete2_basis.count_ops())

83 , 121
Gate counts: OrderedDict([('cx', 61), ('t', 16), ('u3', 15), ('tdg', 12), ('h', 8), ('measure', 6), ('r', 2), ('barrier', 1)])


## Wall time

In [18]:
circuit_LBNL1 = runQuantum(gLR=1,dophisplit=1)
example1 = circuit_optimization( circuit=circuit_LBNL1, slim_level=1, work_register = 'w', cut='high')
circuit_LBNL1_op = example1.slim()

delete1 time: 0.5065131187438965
basis time: 0.018024921417236328
delete2 time: 0.11281800270080566
delete qubit time: 0.0022940635681152344
all time: 0.6440472602844238


## Conclusion
※not included 'measure' and 'barrier'

Original : 486
<br>
↓ -24 (removed adjacent gates pairs)
<br>
↓ -260 (removed unnecessary qubit controls)
<br>
↓ -2 (removed adjacent gates pairs)
<br>
Slimmed one : 114

# PS 2steps

## No_slimming

In [19]:
from sample_algorithm.twostepSim_LBNL import runQuantum
circuit_LBNL1 = runQuantum(gLR=1,dophisplit=1)

In [20]:
print(circuit_LBNL1.depth(), ',', circuit_LBNL1.__len__())
print('Gate counts:', circuit_LBNL1.count_ops())

108 , 158
Gate counts: OrderedDict([('ccx', 74), ('x', 45), ('cu3', 19), ('cx', 10), ('measure', 9), ('barrier', 1)])


In [21]:
circuit_LBNL1_basis = circuit_LBNL1.decompose()
print(circuit_LBNL1_basis.depth(), ',', circuit_LBNL1_basis.__len__())
print('Gate counts:', circuit_LBNL1_basis.count_ops())

770 , 1289
Gate counts: OrderedDict([('cx', 492), ('t', 296), ('tdg', 222), ('h', 148), ('u3', 83), ('u1', 38), ('measure', 9), ('barrier', 1)])


## Removing adjacent gates pairs

In [22]:
from transpiler.optimization import slim
example1 = slim.circuit_optimization( circuit=circuit_LBNL1, slim_level=1, work_register = 'w', cut='high')

In [23]:
circuit_LBNL1 = runQuantum(gLR=1,dophisplit=1)
dag1=circuit_to_dag(circuit_LBNL1)
noU3_dag=example1.u3_transfer_ry(dag1)

In [24]:
delete_dag1 = example1.delete(noU3_dag)
slimmed_delete1 = dag_to_circuit(delete_dag1)

In [25]:
print(slimmed_delete1.depth(), ',', slimmed_delete1.__len__())
print('Gate counts:', slimmed_delete1.count_ops())

98 , 138
Gate counts: OrderedDict([('ccx', 66), ('x', 33), ('cry', 19), ('cx', 10), ('measure', 9), ('barrier', 1)])


In [26]:
slimmed_delete1_basis = slimmed_delete1.decompose()
print(slimmed_delete1_basis.depth(), ',', slimmed_delete1_basis.__len__())
print('Gate counts:', slimmed_delete1_basis.count_ops())

670 , 1119
Gate counts: OrderedDict([('cx', 444), ('t', 264), ('tdg', 198), ('h', 132), ('u3', 71), ('measure', 9), ('barrier', 1)])


## Removing qubit controls

In [27]:
basis_dag1 = example1.basis(delete_dag1)
slimmed_ctrl1 = dag_to_circuit(basis_dag1)

In [28]:
print(slimmed_ctrl1.depth(), ',', slimmed_ctrl1.__len__())
print('Gate counts:', slimmed_ctrl1.count_ops())

95 , 138
Gate counts: OrderedDict([('cx', 46), ('x', 33), ('ccx', 30), ('cry', 17), ('measure', 9), ('ry', 2), ('barrier', 1)])


In [29]:
slimmed_ctrl1_basis = slimmed_ctrl1.decompose()
print(slimmed_ctrl1_basis.depth(), ',', slimmed_ctrl1_basis.__len__())
print('Gate counts:', slimmed_ctrl1_basis.count_ops())

379 , 609
Gate counts: OrderedDict([('cx', 260), ('t', 120), ('tdg', 90), ('u3', 67), ('h', 60), ('measure', 9), ('r', 2), ('barrier', 1)])


## Removing adjacent gates pairs

In [30]:
delete_dag2 = example1.delete(basis_dag1)
slimmed_delete2 = dag_to_circuit(delete_dag2)

In [31]:
print(slimmed_delete2.depth(), ',', slimmed_delete2.__len__())
print('Gate counts:', slimmed_delete2.count_ops())

95 , 132
Gate counts: OrderedDict([('cx', 46), ('ccx', 30), ('x', 27), ('cry', 17), ('measure', 9), ('ry', 2), ('barrier', 1)])


In [32]:
slimmed_delete2_basis = slimmed_delete2.decompose()
print(slimmed_delete2_basis.depth(), ',', slimmed_delete2_basis.__len__())
print('Gate counts:', slimmed_delete2_basis.count_ops())

379 , 603
Gate counts: OrderedDict([('cx', 260), ('t', 120), ('tdg', 90), ('u3', 61), ('h', 60), ('measure', 9), ('r', 2), ('barrier', 1)])


## Wall time

In [34]:
circuit_LBNL1 = runQuantum(gLR=1,dophisplit=1)
example1 = circuit_optimization( circuit=circuit_LBNL1, slim_level=1, work_register = 'w', cut='high')
circuit_LBNL1_op = example1.slim()

delete1 time: 1.2319738864898682
basis time: 0.15824317932128906
delete2 time: 0.6924049854278564
delete qubit time: 0.00737309455871582
all time: 2.1072511672973633


## Conclusion
※not included 'measure' and 'barrier'

Original : 1279
<br>
↓ -170 (removed adjacent gates pairs)
<br>
↓ -510 (removed unnecessary qubit controls)
<br>
↓ -6 (removed adjacent gates pairs)
<br>
Slimmed one : 593